In [ ]:
!python3 -m pip install yacs

In [ ]:
#device = 'cpu'

In [1]:
import os
import numpy as np

# drone flight data
flight_num = 16
data_dir = '/home/Documents/Coding/Python/matching/SLAM/data/4'
data_dir = '/home/Documents/Coding/Python/matching/SLAM/data/12.10.23/16/frames'
data_dir = f'/home/Documents/Coding/Python/matching/SLAM/data/12.10.23/{flight_num}/frames'

paths = [os.path.join(data_dir, name) for name in sorted(os.listdir(data_dir)) if '(1)' not in name]

frame_ids = [int(os.path.basename(path).split('.')[0]) for path in paths]
frame_ids = np.asarray(frame_ids)
sorted_ids = np.argsort(frame_ids)
paths = [paths[idx] for idx in sorted_ids]

len(paths)

6593

In [2]:
from lovio import Lovio, Track             
from utils import read_pos_data, to_homo

from time import time

        
tic = time()

lovio = Lovio(weights="./weights//checkpoints_new_data/LoFTR_39.pt", device='cuda')

poses = []

## set absolute pressure at ground level at the drone starting position
ground_idx = 150
pos = read_pos_data( paths[ground_idx], P0=1013.25 )
P0 = pos['pressure']

# set tracking loop range   
#start_id = 3512
start_id = 4000#600
stop_id = start_id + 1024#4096
step = 1


# read first frame and meta for the track
p_pos = read_pos_data( paths[start_id-1], P0 )

# set track starting point
time0 = p_pos['timestamp']

lat0, lon0, alt0 = p_pos['lat'], p_pos['lon'], 0

track = Track(time0, lat0, lon0)

# main loop
cid = start_id
while cid < stop_id:
    # read next image and meta
    pos = read_pos_data( paths[cid], P0 )
    
     
    # move drone projection from previous frame to current
    lovio.match(p_pos['image'], pos['image'])
    if lovio.M is not None:
        
        proj_dpp = to_homo(p_pos['dpp']).dot(lovio.M.T)
        proj_dpp = proj_dpp[:2] / proj_dpp[2]
        
    else:
        
        proj_dpp = pos['dpp']
    
    # update drone track with new gain
    gain = pos['dpp'] - proj_dpp
    track.update( pos['timestamp'], pos['heading'], gain, pos['altitude'] )
    
    # prepare for the next step
    p_pos = pos.copy()
    pos['image'] = None
    poses.append(pos)
    
    print(cid, end='\r')
    cid += step
    
print((stop_id-start_id)/(time()-tic), 'fps on', lovio.device)    

38.79754145108499 fps on cuda


In [ ]:
'SIFT 17.5 fps on cpu'
'LoFTR 39.5 fps on cuda'
'LoFTR 15.5 fps on cpu'

## track visualization

In [ ]:
from matplotlib import pyplot as plt
import pickle

start, window = 0, len(poses)
rng = range(start, start + window)

glats = np.asarray([pos['lat'] for pos in poses])
glons = np.asarray([pos['lon'] for pos in poses])
dlats, dlons, _ = track.gps.T
plt.plot(dlats[rng], dlons[rng])
plt.plot(glats[rng], glons[rng])
plt.legend(['vio', 'gps'])
plt.show()

gvels = np.asarray([pos['vels'] for pos in poses])
vn, ve = track.vels.T
vx, vy = gvels.T[:2]
plt.scatter(vn[rng], ve[rng])
plt.scatter(vy[rng], vx[rng])
plt.legend(['vio', 'gps'])
plt.show()

data = dict(
            glats=glats,
            glons=glons,
            dlats=dlats,
            dlons=dlons,
           )

with open(f'res/{flight_num}.pickle', 'wb') as f:
    pickle.dump(data, f)